In [10]:
import polars as pl
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense
import helpers.input_processor as ip
import tqdm

In [16]:
#code adapted from: https://towardsdatascience.com/recurrent-neural-nets-for-audio-classification-81cb62327990

def getMFCC(df, data_dir):
    #check if df already has mfcc column
    if 'mfcc' in df.columns:
        return df
    
    progress = tqdm.tqdm(total=df.height, desc='Generating Mel-frequency cepstral coefficients')
    out = df.with_column(
        pl.col('audio_file')
        .apply(lambda x: __function_with_logUpdater(progress, wav_to_mfcc(data_dir + '/' + x)))
        # .apply(lambda x: __function_with_logUpdater(progress, np.array([padding(mfcc,1,400)])))   #not sure what padding does, is it necessary?
        .alias('mfcc')
    )

    out = ip.reorderCols(out)
    return out

def wav_to_mfcc(x, sr=4000):
    waveform, samp_rate = librosa.load(x, sr=sr) #make sure that the correct sample rate is passed as a parameters. if unspecified, the function chooses some default value
    t_start = 0
    t_end = librosa.get_duration(waveform, sr)
    #trim waveform to desired range
    waveform_trimmed = waveform[int(round(t_start*sr)):int(round(t_end*sr))]
    #get Mel-frequency cepstral coefficients
    mfcc = librosa.feature.mfcc(waveform_trimmed)
    return mfcc

def __function_with_logUpdater(progress, func):
    progress.update(1)
    out = func
    return out


In [12]:
data_dir = 'data/raw_training/training_data'
df = ip.loadTrainingData(data_dir, encode_data=True)

loading data from save file:  cache/ingested_data.json


In [13]:
df = df.pipe(getMFCC, data_dir)

Generating Mel-frequency cepstral coefficients: 100%|██████████| 3163/3163 [00:36<00:00, 86.40it/s] 
